In [116]:
import gzip
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
# Metrics for Evaluation of model Accuracy and F1-score
from sklearn.metrics  import f1_score,accuracy_score
#import lightgbm as lgb
#Importing the Decision Tree from scikit-learn library
from sklearn.tree import DecisionTreeClassifier
 
# For splitting of data into train and test set
from sklearn.model_selection import train_test_split

train_data = pd.read_csv('C:/Users/Kedar/Desktop/AML/Zillow/zillow/train_2016.csv')
print (train_data.shape)
train_data.head(10)
property_data = pd.read_csv('C:/Users/Kedar/Desktop/AML/Zillow/zillow/properties_2016.csv')
print (property_data.dtypes)
print (property_data.shape)

(90275, 3)


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (22,32,34,49,55) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


parcelid                          int64
airconditioningtypeid           float64
architecturalstyletypeid        float64
basementsqft                    float64
bathroomcnt                     float64
bedroomcnt                      float64
buildingclasstypeid             float64
buildingqualitytypeid           float64
calculatedbathnbr               float64
decktypeid                      float64
finishedfloor1squarefeet        float64
calculatedfinishedsquarefeet    float64
finishedsquarefeet12            float64
finishedsquarefeet13            float64
finishedsquarefeet15            float64
finishedsquarefeet50            float64
finishedsquarefeet6             float64
fips                            float64
fireplacecnt                    float64
fullbathcnt                     float64
garagecarcnt                    float64
garagetotalsqft                 float64
hashottuborspa                   object
heatingorsystemtypeid           float64
latitude                        float64


## Checking Columns with Object data types

In [117]:
# Replacing Nan for pool count with 0
property_data.poolcnt.fillna(0,inplace = True)

# Replacing hashottuborspa Nan with 0
property_data.hashottuborspa.fillna(0,inplace = True)
# Convert "True" to 1
property_data.hashottuborspa.replace(to_replace = True, value = 1,inplace = True)

# Replacing fireplaceflag Nan with 0
property_data.fireplaceflag.fillna(0, inplace =True)
# Convert "True" to 1
property_data.fireplaceflag.replace(to_replace = True, value = 1,inplace = True)

# Convert Nan for taxdelinquency to 0
property_data.taxdelinquencyflag.fillna(0, inplace =True)
# Convert "Y" to 1
property_data.taxdelinquencyflag.replace(to_replace = 'Y', value = 1,inplace = True)
property_data.head(5)
print (property_data.dtypes)
#print(property_data.hashottuborspa.unique())
#print(property_data.propertycountylandusecode.unique())
#print(property_data.fireplaceflag.unique())
#print(property_data.taxdelinquencyflag.unique())


parcelid                          int64
airconditioningtypeid           float64
architecturalstyletypeid        float64
basementsqft                    float64
bathroomcnt                     float64
bedroomcnt                      float64
buildingclasstypeid             float64
buildingqualitytypeid           float64
calculatedbathnbr               float64
decktypeid                      float64
finishedfloor1squarefeet        float64
calculatedfinishedsquarefeet    float64
finishedsquarefeet12            float64
finishedsquarefeet13            float64
finishedsquarefeet15            float64
finishedsquarefeet50            float64
finishedsquarefeet6             float64
fips                            float64
fireplacecnt                    float64
fullbathcnt                     float64
garagecarcnt                    float64
garagetotalsqft                 float64
hashottuborspa                    int64
heatingorsystemtypeid           float64
latitude                        float64


## Listing non null values by column

In [118]:

property_data.notnull().mean().sort_values()

storytypeid                     0.000544
basementsqft                    0.000545
yardbuildingsqft26              0.000887
architecturalstyletypeid        0.002030
typeconstructiontypeid          0.002260
finishedsquarefeet13            0.002570
buildingclasstypeid             0.004231
decktypeid                      0.005727
finishedsquarefeet6             0.007370
poolsizesum                     0.009366
pooltypeid2                     0.010745
pooltypeid10                    0.012374
taxdelinquencyyear              0.018915
yardbuildingsqft17              0.026918
finishedsquarefeet15            0.063914
finishedfloor1squarefeet        0.067907
finishedsquarefeet50            0.067907
threequarterbathnbr             0.104391
fireplacecnt                    0.104728
pooltypeid7                     0.162621
numberofstories                 0.228482
airconditioningtypeid           0.271846
garagetotalsqft                 0.295880
garagecarcnt                    0.295880
regionidneighbor

## Imputing Data

In [119]:
# If "fireplaceflag" is "1" and "fireplacecnt" is "NaN", we will set "fireplacecnt" to min value of 1".
property_data.loc[(property_data['fireplaceflag'] == 1) & (property_data['fireplacecnt'].isnull()), ['fireplacecnt']] = 1

# If 'fireplacecnt' is "NaN", replace with "0"
property_data.fireplacecnt.fillna(0,inplace = True)

# If "fireplacecnt" is 1 or larger "fireplaceflag" is "0", we will set "fireplaceflag" to "1".
property_data.loc[(property_data['fireplacecnt'] >= 1.0) & (property_data['fireplaceflag']== 0), ['fireplaceflag']] = 1
property_data.fireplaceflag.fillna(0,inplace = True)

# If 'garagecarcnt' is "NaN", replace with "0"
property_data.garagecarcnt.fillna(0,inplace = True)
# If 'garagetotalsqft' is "NaN", replace with "0"
property_data.garagetotalsqft.fillna(0,inplace = True)

# Replace "NaN" with 0 for basementsqft to indicate no basement.
property_data.basementsqft.fillna(0,inplace = True)

# Replace "NaN" with 0 for 'yardbuildingsqft26'
property_data.yardbuildingsqft26.fillna(0,inplace = True)

# Replace 'numberofstories' "NaN"s with "0".
property_data.numberofstories.fillna(1,inplace = True)
# Replace 'yardbuildingsqft17' "NaN"s with "0".
property_data.yardbuildingsqft17.fillna(0,inplace = True)


# Set properties that have a pool but no info on poolsize equal to the median poolsize value.
property_data.loc[property_data.poolcnt== 1, 'poolsizesum'] = property_data.loc[property_data.poolcnt==1, 'poolsizesum'].fillna(property_data[property_data.poolcnt==1].poolsizesum.median())

# "0 pools" = "0 sq ft of pools"
property_data.loc[property_data.poolcnt==0, 'poolsizesum']=0

# "0 pools with a spa/hot tub"
property_data.pooltypeid2.fillna(0,inplace = True)

# "0 pools without a hot tub"
property_data.pooltypeid7.fillna(0,inplace = True)

# Replace NaN with 1
property_data.unitcnt.fillna(1,inplace = True)

property_data.notnull().mean().sort_values()


# Replace "NaN" "calculatedfinishedsquarefeet" values with mean.
property_data['calculatedfinishedsquarefeet'].fillna((property_data['calculatedfinishedsquarefeet'].mean()), inplace=True)

# Replace "NaN" "finishedsquarefeet15" values with calculatedfinishedsquarefeet.
property_data.loc[property_data['finishedsquarefeet15'].isnull(),'finishedsquarefeet15'] = property_data['calculatedfinishedsquarefeet']
property_data.notnull().mean().sort_values()


# Replace with max value_count
airconditioning = property_data['airconditioningtypeid'].value_counts().argmax()
#print(airconditioning)
#print(property_data['airconditioningtypeid'].value_counts())
property_data.airconditioningtypeid.fillna(5,inplace = True)
# Replace with max value_count
buildingqual = property_data['buildingqualitytypeid'].value_counts().argmax()
#print(buildingqual)
#print(property_data['buildingqualitytypeid'].value_counts())
property_data['buildingqualitytypeid'] = property_data['buildingqualitytypeid'].fillna(buildingqual)

#print(property_data['propertyzoningdesc'].value_counts())

#print(property_data['heatingorsystemtypeid'].value_counts())
# Fill in "NaN" "heatingorsystemtypeid" with most common
heatingorsystemtypeid = property_data['heatingorsystemtypeid'].value_counts().argmax()
property_data['heatingorsystemtypeid'] = property_data['heatingorsystemtypeid'].fillna(heatingorsystemtypeid)
#print(property_data['heatingorsystemtypeid'].value_counts())



#print(property_data['calculatedbathnbr'].value_counts())
# Fill in "NaN" "calculatedbathnbr" with most common
calculatedbathnbr = property_data['calculatedbathnbr'].value_counts().argmax()
property_data['calculatedbathnbr'] = property_data['calculatedbathnbr'].fillna(calculatedbathnbr)
#print(property_data['calculatedbathnbr'].value_counts())


#print(property_data['fullbathcnt'].value_counts())
# Fill in "NaN" "fullbathcnt" with most common
fullbathcnt = property_data['fullbathcnt'].value_counts().argmax()
property_data['fullbathcnt'] = property_data['fullbathcnt'].fillna(fullbathcnt)
#print(property_data['fullbathcnt'].value_counts())

#print(property_data['threequarterbathnbr'].value_counts())
# Fill in "NaN" "threequarterbathnbr" with most common
threequarterbathnbr = property_data['threequarterbathnbr'].value_counts().argmax()
property_data['threequarterbathnbr'] = property_data['threequarterbathnbr'].fillna(threequarterbathnbr)
#print(property_data['threequarterbathnbr'].value_counts())


# Fill in "NaN" "propertyzoningdesc" with most common
propertyzoningdesc = property_data['propertyzoningdesc'].value_counts().argmax()
property_data['propertyzoningdesc'] = property_data['propertyzoningdesc'].fillna(propertyzoningdesc)

# replace with mean
property_data['taxvaluedollarcnt'].fillna((property_data['taxvaluedollarcnt'].mean()), inplace=True)
property_data['lotsizesquarefeet'].fillna((property_data['lotsizesquarefeet'].mean()), inplace=True)

# for properties with no land replace Nan with 0
property_data.landtaxvaluedollarcnt.fillna(0,inplace = True)
# for properties with no structure replace Nan with 0
property_data.structuretaxvaluedollarcnt.fillna(0,inplace = True)

property_data['taxamount'].fillna((property_data['taxamount'].mean()), inplace=True)



In [120]:
# Fill in "NaN" "yearbuilt" with most common
yearbuilt = property_data['yearbuilt'].value_counts().argmax()
property_data['yearbuilt'] = property_data['yearbuilt'].fillna(yearbuilt)

# Fill in "fips" "NaN"s
fips = property_data['fips'].value_counts().argmax()
property_data['fips'] = property_data['fips'].fillna(fips)

# Fill in "propertylandusetypeid" "NaN"s
propertylandusetypeid = property_data['propertylandusetypeid'].value_counts().argmax()
property_data['propertylandusetypeid'] = property_data['propertylandusetypeid'].fillna(propertylandusetypeid)

# Drop 'regionidcounty'
property_data.drop('regionidcounty', axis=1, inplace=True)

# Fill in "latitude" "NaN"s
latitude = property_data['latitude'].value_counts().argmax()
property_data['latitude'] = property_data['latitude'].fillna(latitude)

# Fill in "longitude" "NaN"s
longitude = property_data['longitude'].value_counts().argmax()
property_data['longitude'] = property_data['longitude'].fillna(longitude)

# Fill in "censustractandblock" "NaN"s
censustractandblock = property_data['censustractandblock'].value_counts().argmax()
property_data['censustractandblock'] = property_data['censustractandblock'].fillna(censustractandblock)

# Fill in "assessmentyear" "NaN"s
assessmentyear = property_data['assessmentyear'].value_counts().argmax()
property_data['assessmentyear'] = property_data['assessmentyear'].fillna(assessmentyear)

# Fill in "bedroomcnt" "NaN"s
bedroomcnt = property_data['bedroomcnt'].value_counts().argmax()
property_data['bedroomcnt'] = property_data['bedroomcnt'].fillna(bedroomcnt)

# Fill in "bathroomcnt" "NaN"s
bathroomcnt = property_data['bathroomcnt'].value_counts().argmax()
property_data['bathroomcnt'] = property_data['bathroomcnt'].fillna(bathroomcnt)

# Fill in "roomcnt" "NaN"s
roomcnt = property_data['roomcnt'].value_counts().argmax()
property_data['roomcnt'] = property_data['roomcnt'].fillna(roomcnt)

# Fill in "propertycountylandusecode" "NaN"s
propertycountylandusecode = property_data['propertycountylandusecode'].value_counts().argmax()
property_data['propertycountylandusecode'] = property_data['propertycountylandusecode'].fillna(propertycountylandusecode)

# Fill in "regionidzip " "NaN"s
regionidzip = property_data['regionidzip'].value_counts().argmax()
property_data['regionidzip'] = property_data['regionidzip'].fillna(regionidzip)





## Dropping unwanted columns 

In [121]:
# Drop unwanted columns 
property_data.drop('storytypeid', axis=1, inplace=True)
property_data.drop('architecturalstyletypeid', axis=1, inplace=True)
property_data.drop('typeconstructiontypeid', axis=1, inplace=True)
property_data.drop('finishedsquarefeet13', axis=1, inplace=True)
property_data.drop('buildingclasstypeid', axis=1, inplace=True)
property_data.drop('decktypeid', axis=1, inplace=True)
property_data.drop('finishedsquarefeet6', axis=1, inplace=True)
property_data.drop('pooltypeid10', axis=1, inplace=True)                     
property_data.drop('finishedsquarefeet12', axis=1, inplace=True)
property_data.drop('finishedfloor1squarefeet', axis=1, inplace=True) 
property_data.drop('rawcensustractandblock', axis=1, inplace=True)
property_data.drop('regionidcity', axis=1, inplace=True)
property_data.drop('taxdelinquencyyear', axis=1, inplace=True)
property_data.drop('finishedsquarefeet50', axis=1, inplace=True)
property_data.drop('regionidneighborhood', axis=1, inplace=True)

property_data.notnull().mean().sort_values()
print( property_data.shape)

(2985217, 42)


## Merging Data Sources to create final data

In [128]:
final_data = train_data.merge(property_data, on='parcelid',how='left')
final_data.head(5)
final_data.dtypes

parcelid                          int64
logerror                        float64
transactiondate                  object
airconditioningtypeid           float64
basementsqft                    float64
bathroomcnt                     float64
bedroomcnt                      float64
buildingqualitytypeid           float64
calculatedbathnbr               float64
calculatedfinishedsquarefeet    float64
finishedsquarefeet15            float64
fips                            float64
fireplacecnt                    float64
fullbathcnt                     float64
garagecarcnt                    float64
garagetotalsqft                 float64
hashottuborspa                    int64
heatingorsystemtypeid           float64
latitude                        float64
longitude                       float64
lotsizesquarefeet               float64
poolcnt                         float64
poolsizesum                     float64
pooltypeid2                     float64
pooltypeid7                     float64


## Creating Training and Test Data Split of 49-51%

In [129]:
X = final_data.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 'propertycountylandusecode'], axis=1)
y = final_data['logerror'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.51, random_state=2)

print(X.shape, y.shape)

(90275, 39) (90275,)


## Performing Regression

In [130]:
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor

In [131]:
# LGB
model1 = lgb.LGBMRegressor(objective='regression', num_leaves=60, learning_rate=0.002, boosting_type ='gbdt', min_data_in_leaf =500,min_sum_hessian_in_leaf =1, feature_fraction =0.5)
model1.fit(X_test, y_test)
model1_prediction = model1.predict(X_test)
lgb_mae = mean_absolute_error(y_test, model1_prediction)

#ARD
model2 = linear_model.ARDRegression(n_iter=500,fit_intercept=False,normalize=False)
model2.fit(X_test, y_test)
model2_prediction = model2.predict(X_test)
ard_mae = mean_absolute_error(y_test, model2_prediction)


#SVM
#model3 = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))

#model3.fit(X_test, y_test)
#model3_prediction = model3.predict(X_test)
#svm_mae = mean_absolute_error(y_test, model3_prediction)

# Lars
model3 = linear_model.Lars(n_nonzero_coefs=1, normalize=False)
model3.fit(X_test, y_test)
model3_prediction = model3.predict(X_test)
lars_mae = mean_absolute_error(y_test, model3_prediction)

#Linear

model4 = LinearRegression()
model4.fit(X_test, y_test)
model4_prediction = model4.predict(X_test)
linear_mae = mean_absolute_error(y_test, model4_prediction)


#Laso
#model5 = linear_model.Lasso(alpha=0.1)
#model5.fit(X_train, y_train)
#model5_prediction = model5.predict(X_train)
#laso_mae = mean_absolute_error(y_train, model5_prediction)

[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=1, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=1


In [132]:
print("LGBM gives MAE of : ", lgb_mae)
print("ARD gives MAE of : ", ard_mae)
print("Lars gives MAE of : ", lars_mae)
print("Linear gives MAE of : ", linear_mae)
#print("Laso gives MAE of : ", laso_mae)


LGBM gives MAE of :  0.06809921391181507
ARD gives MAE of :  0.0683731326426446
Lars gives MAE of :  0.0684129672446621
Linear gives MAE of :  0.06835902416266693


## Exporting model using Pickle

In [133]:
import pickle
# save the model to disk
with open('C:/Users/Kedar/Desktop/AML/Zillow/lgb.model','wb') as lgb:
    pickle.dump(model1_prediction,lgb)
with open('C:/Users/Kedar/Desktop/AML/Zillow/ard.model','wb') as ard:
    pickle.dump(model2_prediction,ard)
with open('C:/Users/Kedar/Desktop/AML/Zillow/zillow.model','wb') as zil:
    pickle.dump(model2_prediction,zil)